In [1]:
# -- dependencies
import pandas as pd
import pyreadstat
import os

In [5]:
# -- defining paths as variables for all files needed (substitute with your local file paths)
nhanes_ghb_path = r"data\raw\NHANES_GHB_L_21-23.xpt"
nhanes_glu_path = r"data\raw\NHANES_GLU_L_21-23.xpt"
nhanes_ins_path = r"data\raw\NHANES_INS_L_21-23.xpt"
nhanes_demo_path = r"data\raw\NHANES_DEMO_L_21-23.xpt"
nhanes_exam_path = r"data\raw\NHANES_BMX_L_21-23.xpt"

In [3]:
# -- function to read xpt file and return df (with info to call)
def read_xpt(path, df_name):
    try:
        df, meta = pyreadstat.read_xport(path)
        globals()[df_name] = df
        print(f"DataFrame '{df_name} created successfully.")
    except UnicodeDecodeError as e:                                    # section added to handle unicode error in nhanes_demo dataset
        print(f"UnicodeDecodeError encountered: {e}")
        print(f"... retrying with 'latin1' encoding...")
        try: 
            df, meta = pyreadstat.read_xport(path, encoding="latin1")
            globals()[df_name] = df
            print(f"DataFrame '{df_name} created successfully.")
        except Exception as e2:
            print(f"Failed to read file with 'latin1' encoding: {e2}")
    except Exception as e:
        print(f"Error reading file {path}: {e}")

# call this function using:
# path = nhanes_ghb_path
# read_xpt(path, "df_ghb")
# print(df_ghb)

In [4]:
# -- turning all files into dataframes & basic cleaning to only get desired fields

# for the glycohemoglobin (a1c) data
path = nhanes_ghb_path
read_xpt(path, "df_ghb")
df_ghb = df_ghb[['SEQN', 'WTPH2YR', 'LBXGH']]
df_ghb = df_ghb.rename(columns={
    'SEQN': 'seqn_id',
    'LBXGH': 'a1c_perc', 
    'WTPH2YR': 'ghb_weight'
    })
print(df_ghb)

# for the fasting glucose data
path = nhanes_glu_path
read_xpt(path, "df_glu")
df_glu = df_glu[['SEQN', 'LBXGLU', 'WTSAF2YR']]
df_glu = df_glu.rename(columns={
    'SEQN': 'seqn_id',
    'LBXGLU': 'f_glucose_mgdl',
    'WTSAF2YR': 'fasting_weight'     # the same weight is used for both fasting glucose and fasting insulin
    })
print(df_glu)

# for the insulin data
path = nhanes_ins_path
read_xpt(path, "df_ins")
df_ins = df_ins[['SEQN', 'LBXIN']]
df_ins = df_ins.rename(columns={
    'SEQN': 'seqn_id',
    'LBXIN': 'f_insulin_uuml',
    })
print(df_ins)

# for the demographic (age) data
path = nhanes_demo_path
read_xpt(path, "df_demo")
# df_demo = df_demo[['SEQN', 'RIDAGEYR', 'INDFMPIR', 'DMDEDUC2', 'WTINT2YR', 'WTMEC2YR']]       # optional / future
df_demo = df_demo[['SEQN', 'RIDAGEYR', 'WTINT2YR', 'WTMEC2YR']]
df_demo = df_demo.rename(columns={
    'SEQN': 'seqn_id',
    'RIDAGEYR': 'age_yrs',
    # 'INDFMPIR': 'income_poverty_ratio',
    # 'DMDEDUC2': 'edu_level',
    'WTINT2YR': 'demo_weight',
    'WTMEC2YR': 'exam_weight'       # the exam weight is stored with the demographic data
    })
print(df_demo)

# for the exam (bmi) data
path = nhanes_exam_path
read_xpt(path, "df_exam")
df_exam = df_exam[['SEQN', 'BMXBMI']]
df_exam = df_exam.rename(columns={
    'SEQN': 'seqn_id',
    'BMXBMI': 'bmi_kg'
    })
print(df_exam)


DataFrame 'df_ghb created successfully.
       seqn_id    ghb_weight  a1c_perc
0     130378.0  56042.129410       5.6
1     130379.0  37435.705647       5.6
2     130380.0  85328.844519       6.2
3     130386.0  44526.214135       5.1
4     130387.0  22746.296353       5.9
...        ...           ...       ...
7194  142305.0  49710.929024       6.0
7195  142307.0  68994.175834       6.2
7196  142308.0      0.000000       NaN
7197  142309.0  46284.416719       5.2
7198  142310.0  53250.820372       5.3

[7199 rows x 3 columns]
DataFrame 'df_glu created successfully.
       seqn_id  f_glucose_mgdl  fasting_weight
0     130378.0           113.0   120025.308444
1     130379.0            99.0        0.000000
2     130380.0           156.0   145090.773569
3     130386.0           100.0    82599.618089
4     130394.0            88.0   100420.348913
...        ...             ...             ...
3991  142301.0           110.0    31123.370660
3992  142303.0           160.0   109582.336415
3993

In [6]:
# -- combining all of the dataframes into one & cleaning up
# merging all (with seqn as key)
combined_df = pd.merge(df_ghb, df_glu, on='seqn_id', how='inner')
combined_df = pd.merge(combined_df, df_ins, on='seqn_id', how='inner')
combined_df = pd.merge(combined_df, df_demo, on='seqn_id', how='inner')
combined_df = pd.merge(combined_df, df_exam, on='seqn_id', how='inner')

# reordering columns for readability 
combined_df = combined_df[['seqn_id','a1c_perc', 'f_glucose_mgdl', 'f_insulin_uuml', 'age_yrs', 'bmi_kg',
                           'ghb_weight', 'fasting_weight', 'demo_weight', 'exam_weight']]

# dropping nulls so that we are only looking at participants with all values 
combined_df = combined_df.dropna(subset=['a1c_perc'])
combined_df = combined_df.dropna(subset=['f_glucose_mgdl'])
combined_df = combined_df.dropna(subset=['f_insulin_uuml'])
combined_df = combined_df.dropna(subset=['age_yrs'])
combined_df = combined_df.dropna(subset=['bmi_kg'])

# return df
clean_df = combined_df
print(clean_df)

       seqn_id  a1c_perc  f_glucose_mgdl  f_insulin_uuml  age_yrs  bmi_kg  \
0     130378.0       5.6           113.0           15.53     43.0    27.0   
1     130379.0       5.6            99.0           19.91     66.0    33.5   
2     130380.0       6.2           156.0           16.33     44.0    29.7   
3     130386.0       5.1           100.0           11.38     34.0    30.2   
4     130394.0       4.8            88.0            7.20     51.0    24.4   
...        ...       ...             ...             ...      ...     ...   
3990  142300.0       5.1            95.0           10.35     46.0    32.6   
3991  142301.0       5.6           110.0            5.62     80.0    30.5   
3992  142303.0       8.1           160.0           17.09     69.0    27.9   
3993  142305.0       6.0           132.0           24.86     76.0    26.4   
3995  142309.0       5.2            96.0            6.22     40.0    25.5   

        ghb_weight  fasting_weight   demo_weight   exam_weight  
0     5604

In [7]:
# reviewing 
# clean_df.describe()      # reviewing basic stats for each column
clean_df.info()          # gives number of non-null values 

# check columns for '0' values (this validation could find '0' values that indicate missing data but were not dropped with the nulls)
# the NHANES documentation indicates that all of our core data (a1c, glucose, insulin, age, and bmi) is free of '0' values (but the weights are not) 
total_zeros = (clean_df == 0).sum()
print("Missing data ('0' values): ", total_zeros)

# -- rationale / notes  
# we only want to use the records that have all core values AND the selected weight (and we have not selected a weight yet) 
# ... so, e.g., if we utilize a weight with any '0' values, we need to also filter those records out (as they will not represent total pop) 
# ... looks like fasting_weight has a number of '0' values, despite having recorded values for both glucose and insulin (& we dropped nulls above ^) 
# ... the documentation says that a '0' for this weight (WTSAF2YR) means "No Lab Result or Not Fasting for 8 to <24 hours" ...
# ... so it seems like survey administrators identified/removed folks from fasting pop weight who had fasting tests done (for one reason or another)

<class 'pandas.core.frame.DataFrame'>
Index: 3441 entries, 0 to 3995
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   seqn_id         3441 non-null   float64
 1   a1c_perc        3441 non-null   float64
 2   f_glucose_mgdl  3441 non-null   float64
 3   f_insulin_uuml  3441 non-null   float64
 4   age_yrs         3441 non-null   float64
 5   bmi_kg          3441 non-null   float64
 6   ghb_weight      3441 non-null   float64
 7   fasting_weight  3441 non-null   float64
 8   demo_weight     3441 non-null   float64
 9   exam_weight     3441 non-null   float64
dtypes: float64(10)
memory usage: 295.7 KB
Missing data ('0' values):  seqn_id             0
a1c_perc            0
f_glucose_mgdl      0
f_insulin_uuml      0
age_yrs             0
bmi_kg              0
ghb_weight          0
fasting_weight    285
demo_weight         0
exam_weight         0
dtype: int64


In [8]:
# -- evaluate the population weights to see which is the best fit for normalization
    # the NHANES documentation indicates that, when combining datasets, a single weight should be used, and it should be the most restrictive
    # ... so we want to consider which of the original weights has the most overlap with our filtered data 
    # ... which means we need to calculate the count of population weights from each original dataframe for comparison (and scale up to the sum)
    # (the NHANES documentation does present the total records with/without missing values, but we will show/validate the calculations here) 

# -- count of records from our new filtered dataframe
# (this is the number we have filtered down to - need to normalize the weights based on this!)
count_filtered = clean_df['seqn_id'].shape[0]
print("Count of filtered records (core data): ", count_filtered)

# -- counts of population weights from each original dataframe
# (checking the count of weights that have values greater than 0 will exclude any missing or null values)
# ghb weight
count_ghb = df_ghb[(df_ghb['ghb_weight'] > 0)].shape[0]
print("Count of ghb_weight: ", count_ghb)
diff_ghb = round(((count_ghb - count_filtered) / count_ghb), 3)
print("Percent filtered ghb_weight: ", diff_ghb)

# fasting weight
count_fasting = df_glu[(df_glu['fasting_weight'] > 0)].shape[0]
print("Count of fasting_weight: ", count_fasting)
diff_fasting = round(((count_fasting - (count_filtered - 285)) / count_fasting), 3)    # added -285 to remove the '0' records that would need to be filtered out
print("Percent filtered fasting_weight: ", diff_fasting)

# demo weight
count_demo = df_demo[(df_demo['demo_weight'] > 0)].shape[0]
print("Count of demo_weight: ", count_demo)
diff_demo = round(((count_demo - count_filtered) / count_demo), 3)
print("Percent filtered demo_weight: ", diff_demo)

# exam weight
count_exam = df_demo[(df_demo['exam_weight'] > 0)].shape[0]
print("Count of exam_weight: ", count_exam)
diff_exam = round(((count_exam - count_filtered) / count_exam), 3)
print("Percent filtered exam_weight: ", diff_exam)

# -- find minimum weight
best_weight = min(diff_ghb, diff_fasting, diff_demo, diff_exam)
print("The best weight for normalization is the smallest percentage loss from original after filtering: ", best_weight)

# summary: normalizing to fasting_weight will only result in 6.1% loss of records from original weight count

Count of filtered records (core data):  3441
Count of ghb_weight:  6750
Percent filtered ghb_weight:  0.49
Count of fasting_weight:  3361
Percent filtered fasting_weight:  0.061
Count of demo_weight:  11933
Percent filtered demo_weight:  0.712
Count of exam_weight:  8860
Percent filtered exam_weight:  0.612
The best weight for normalization is the smallest percentage loss from original after filtering:  0.061


In [9]:
# -- adding a new, normalized weight based on the filtered core data

# drop records where fasting_weight equal to 0
cleaning_df = clean_df[clean_df['fasting_weight'] != 0].copy()

# calculate the sum of the fasting weight in the filtered data
filtered_weight_sum = cleaning_df['fasting_weight'].sum()

# get total population using original fasting_weight (before filtered to core data)
# since no records dropped from this value yet, and it includes all lab participants, this sum is best view of total population 
total_population_sum = df_glu['fasting_weight'].sum()

# factor to scale up filtered weight
scale_factor = total_population_sum / filtered_weight_sum

# calc normalized weight (scale the filtered fasting weight to be proportionate to the total population from original fasting weight)
cleaning_df['normalized_weight'] = cleaning_df['fasting_weight'] * scale_factor

# drop unneeded weights
cleaned_df = cleaning_df.drop(columns=['ghb_weight', 'demo_weight', 'exam_weight' 
                                  , 'fasting_weight'                                                # optional, keep to review
                                 ])

# rename the dataframe before load
normalized_df = cleaned_df

# print results
print("Total Population: ", total_population_sum)
print("Filtered Weight Sum: ", filtered_weight_sum)
print("Scale Factor: ", scale_factor)
print("Normalized Fasting Weight Sum: ", normalized_df['normalized_weight'].sum())
print(normalized_df)

Total Population:  279992595.93949234
Filtered Weight Sum:  264249300.74824518
Scale Factor:  1.0595774336835277
Normalized Fasting Weight Sum:  279992595.93949234
       seqn_id  a1c_perc  f_glucose_mgdl  f_insulin_uuml  age_yrs  bmi_kg  \
0     130378.0       5.6           113.0           15.53     43.0    27.0   
2     130380.0       6.2           156.0           16.33     44.0    29.7   
3     130386.0       5.1           100.0           11.38     34.0    30.2   
4     130394.0       4.8            88.0            7.20     51.0    24.4   
6     130396.0       5.0           104.0            4.11     56.0    27.3   
...        ...       ...             ...             ...      ...     ...   
3990  142300.0       5.1            95.0           10.35     46.0    32.6   
3991  142301.0       5.6           110.0            5.62     80.0    30.5   
3992  142303.0       8.1           160.0           17.09     69.0    27.9   
3993  142305.0       6.0           132.0           24.86     76.0 

In [10]:
# -- adding in calculations to indicate health or severity of lab results

# interpreting a1c_perc according to a1c thresholds
def interpret_a1c(a1c):
    if a1c < 5.7:
        return 'normal'
    elif 5.7 <= a1c < 6.5:
        return 'prediabetes'
    elif a1c >= 6.5:
        return 'diabetes'
    else: 
        return 'unknown'

normalized_df['a1c_threshold'] = normalized_df['a1c_perc'].apply(interpret_a1c)

# calculating and interpreting insulin sensitivity using HOMA-IR formula
normalized_df['homa_ir'] = (normalized_df['f_glucose_mgdl'] * normalized_df['f_insulin_uuml']) / 405

def interpret_homa(homa_ir):
    if homa_ir < 2.0:                     # could add 'heightened sensitivity' as less than 1, but this is simple and consistent with a1c thresholds
        return 'normal sensitivity'
    elif 2.0 <= homa_ir < 3.0:
        return 'early resistance'
    elif homa_ir >= 3.0:
        return 'significant resistance'
    else:
        return 'unknown'

normalized_df['insulin_sensitivity'] = normalized_df['homa_ir'].apply(interpret_homa)

# cohorting by age group (decade intervals)
def age_cohort(age):
    if 0 <= age < 12:
        return '12 years or less'
    elif 12 <= age < 20:
        return '12 to 20 years'
    elif 20 <= age < 30:
        return '20 to 30 years'
    elif 30 <= age < 40:
        return '30 to 40 years'
    elif 40 <= age < 50:
        return '40 to 50 years'
    elif 50 <= age < 60:
        return '50 to 60 years'
    elif 60 <= age < 70:
        return '60 to 70 years'
    elif 70 <= age < 80:
        return '70 to 80 years'
    elif age >= 80:
        return '80 years or older'        # NHANES records all participants over 80 as just '80' 
    else:
        return 'unknown'

normalized_df['age_cohort'] = normalized_df['age_yrs'].apply(age_cohort)

# interpreting BMI u
def interpret_bmi(bmi):
    if bmi < 18.5:
        return 'A: underweight'
    elif 18.5 <= bmi < 25:
        return 'B: normal weight'
    elif 25 <= bmi < 30:
        return 'C: overweight'
    elif 30 <= bmi < 35:
        return 'D: obesity class 1'
    elif 35 <= bmi < 40:
        return 'E: obesity class 2'
    elif bmi >= 40:
        return 'F: extreme obesity class 3'
    else:
        return 'Unknown'

normalized_df['bmi_threshold'] = normalized_df['bmi_kg'].apply(interpret_bmi)


nhanes_all = normalized_df
# print(normalized_df)
print(nhanes_all)

       seqn_id  a1c_perc  f_glucose_mgdl  f_insulin_uuml  age_yrs  bmi_kg  \
0     130378.0       5.6           113.0           15.53     43.0    27.0   
2     130380.0       6.2           156.0           16.33     44.0    29.7   
3     130386.0       5.1           100.0           11.38     34.0    30.2   
4     130394.0       4.8            88.0            7.20     51.0    24.4   
6     130396.0       5.0           104.0            4.11     56.0    27.3   
...        ...       ...             ...             ...      ...     ...   
3990  142300.0       5.1            95.0           10.35     46.0    32.6   
3991  142301.0       5.6           110.0            5.62     80.0    30.5   
3992  142303.0       8.1           160.0           17.09     69.0    27.9   
3993  142305.0       6.0           132.0           24.86     76.0    26.4   
3995  142309.0       5.2            96.0            6.22     40.0    25.5   

      normalized_weight a1c_threshold   homa_ir     insulin_sensitivity  \


In [11]:
# optional
# drop columns that won't be needed, reorder the existing columns, add a static total_population column for easier analysis 
nhanes_final = nhanes_all.drop(columns = ['f_glucose_mgdl', 'f_insulin_uuml'])
nhanes_final = nhanes_final[['seqn_id', 'a1c_perc', 'a1c_threshold', 'homa_ir', 'insulin_sensitivity', 
                             'age_yrs', 'age_cohort', 'bmi_kg', 'bmi_threshold', 'normalized_weight']]
nhanes_final['total_pop'] = total_population_sum 

print(nhanes_final)

       seqn_id  a1c_perc a1c_threshold   homa_ir     insulin_sensitivity  \
0     130378.0       5.6        normal  4.333062  significant resistance   
2     130380.0       6.2   prediabetes  6.290074  significant resistance   
3     130386.0       5.1        normal  2.809877        early resistance   
4     130394.0       4.8        normal  1.564444      normal sensitivity   
6     130396.0       5.0        normal  1.055407      normal sensitivity   
...        ...       ...           ...       ...                     ...   
3990  142300.0       5.1        normal  2.427778        early resistance   
3991  142301.0       5.6        normal  1.526420      normal sensitivity   
3992  142303.0       8.1      diabetes  6.751605  significant resistance   
3993  142305.0       6.0   prediabetes  8.102519  significant resistance   
3995  142309.0       5.2        normal  1.474370      normal sensitivity   

      age_yrs         age_cohort  bmi_kg       bmi_threshold  \
0        43.0     40 to

In [12]:
# -- reviewing finalized data 
# nhanes_all.describe()
# nhanes_all.info()

nhanes_final.describe()
# nhanes_final.info()

,seqn_id,a1c_perc,homa_ir,age_yrs,bmi_kg,normalized_weight,total_pop
count,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3.156000e+03
mean,136376.736692,5.700729,4.007540,49.643536,29.224588,88717.552579,2.799926e+08
std,3453.024305,1.015061,9.398656,20.167995,7.447206,65852.953095,5.961409e-08
min,130378.000000,3.200000,0.066543,12.000000,14.000000,12141.892446,2.799926e+08
25%,133389.750000,5.200000,1.470667,33.000000,24.000000,44043.536821,2.799926e+08
50%,136401.500000,5.500000,2.436741,53.000000,27.900000,69787.883312,2.799926e+08
75%,139351.250000,5.800000,4.098883,66.000000,33.100000,111879.235148,2.799926e+08
max,142309.000000,13.700000,227.243704,80.000000,74.800000,595400.325846,2.799926e+08


In [13]:
# -- data loading dependencies
from sqlalchemy import create_engine
# import os
import pyodbc
import psycopg2

# -- database connection
pad = 'postgres'
uid = 'postgres'
server = 'localhost'

# -- loading data
def load_data(df):
    try:
        rows_imported = 0
        len_df = len(df)
        engine = create_engine(f'postgresql://{uid}:{pad}@{server}:{port}/{database_name}')
        print(f'importing rows {rows_imported} to {rows_imported + len_df}... for table')
        df.to_sql(table_name, engine, if_exists='replace', index=False) # creates table
        rows_imported += len_df
        print("data imported successfully 🚀")
    except Exception as e:
        print("data load error: " + str(e))


In [ ]:
# -- loading the data (final_df)
uid = 'postgres'                # db username
pad = 'postgres'                # password for user
server = 'localhost'            # host/server address, localhost if local
port = '5432'                   # port (usually 5432)
database_name = 'postgres'      # database name 
# ------------------------- 
table_name = 'nhanes_data'      # new table name
load_data(nhanes_final)         # dataframe to load